## 1. Import du Dataset from Roboflow

In [12]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GBbY0P8ZuDM8EocauwAB")
project = rf.workspace("visionqai").project("detection-defauts-3ikxt")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...


## 2. Import du Modèle from Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/best.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Prédire sur ton dossier de validation
results = model.predict(source='/content/Détection-défauts-2/valid/images', save=True)



image 1/20 /content/Détection-défauts-2/valid/images/IMG_0884_JPG.rf.b26491192cd331bb7e70573be33144db.jpg: 640x640 2 cable 2s, 24.5ms
image 2/20 /content/Détection-défauts-2/valid/images/IMG_0891_JPG.rf.82ac1219ad20cee78e0758d6a96e0e99.jpg: 640x640 1 cable 2, 15.3ms
image 3/20 /content/Détection-défauts-2/valid/images/photo-1761661075325_jpg.rf.ad5962f4df803ed77c3b63140114418e.jpg: 640x640 1 cable 2, 42.0ms
image 4/20 /content/Détection-défauts-2/valid/images/photo-1761661091891_jpg.rf.19e25e4c5f7dc226851b562525081395.jpg: 640x640 1 cable 2, 37.0ms
image 5/20 /content/Détection-défauts-2/valid/images/photo-1761661325771_jpg.rf.5b9d65fbd9258091884393ca6595982e.jpg: 640x640 1 cable 2, 26.1ms
image 6/20 /content/Détection-défauts-2/valid/images/photo-1761661484246_jpg.rf.b577cc05964bd5f09807e12977624af8.jpg: 640x640 1 cable 2, 44.6ms
image 7/20 /content/Détection-défauts-2/valid/images/photo-1761661508569_jpg.rf.9231e77d6c659237c1b01cf2b5565f25.jpg: 640x640 1 cable 2, 41.6ms
image 8/20 /

## 3. Génération des Masques

In [15]:
import os
import cv2

# Définir les dossiers à traiter
datasets = {
    "train": "/content/Détection-défauts-2/train/images",
    "valid": "/content/Détection-défauts-2/valid/images"
}

# Boucle sur train et valid
for split, images_folder in datasets.items():
    masks_folder = f"/content/dataset_masks/{split}"
    os.makedirs(masks_folder, exist_ok=True)

    for img_name in os.listdir(images_folder):
        img_path = os.path.join(images_folder, img_name)
        results = model(img_path)

        # Vérifier s'il y a des masques détectés
        if results[0].masks is not None and len(results[0].masks.data) > 0:
            # Récupération du masque
            mask = results[0].masks.data[0].cpu().numpy()
            mask = (mask > 0.5).astype("uint8") * 255

            # Enregistrement du masque
            mask_path = os.path.join(masks_folder, img_name.replace(".jpg","_mask.png"))
            cv2.imwrite(mask_path, mask)

            print(f"Masque généré : {mask_path}")
        else:
            print(f"Aucun masque détecté pour : {img_name}")


image 1/1 /content/Détection-défauts-2/train/images/photo-1761661286886_jpg.rf.91ce9f4813d2ab9989a1bb135d394074.jpg: 640x640 1 cable 2, 21.0ms
Speed: 5.2ms preprocess, 21.0ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)
Masque généré : /content/dataset_masks/train/photo-1761661286886_jpg.rf.91ce9f4813d2ab9989a1bb135d394074_mask.png

image 1/1 /content/Détection-défauts-2/train/images/photo-1761661652765_jpg.rf.8cb2a38366ebae840125f6989e36ecb7.jpg: 640x640 1 cable 2, 10.6ms
Speed: 2.0ms preprocess, 10.6ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)
Masque généré : /content/dataset_masks/train/photo-1761661652765_jpg.rf.8cb2a38366ebae840125f6989e36ecb7_mask.png

image 1/1 /content/Détection-défauts-2/train/images/photo-1761661794293_jpg.rf.24b3af4bfd2689e88f4aee4107d72d3d.jpg: 640x640 1 cable 2, 25.5ms
Speed: 6.9ms preprocess, 25.5ms inference, 11.2ms postprocess per image at shape (1, 3, 640, 640)
Masque généré : /content/dataset_masks/train/pho

## 4. Nettoyage des Masques

In [16]:
from skimage import morphology
from scipy import ndimage as ndi

def preprocess_mask(mask):
    """Nettoyage du masque : fermeture, suppression petits objets, remplissage des trous"""
    mask = mask > 0
    mask = morphology.binary_closing(mask, morphology.disk(2))
    mask = morphology.remove_small_objects(mask, min_size=50)
    mask = ndi.binary_fill_holes(mask)
    return mask.astype(bool)

## 5. Squelettisation

In [17]:
from skimage.morphology import skeletonize

def skeletonize_mask(mask):
    """Squelettisation du masque"""
    return skeletonize(mask)

## 6. Construction du Graphe et Calcul de la Longueur en Pixels

In [18]:
import networkx as nx
import math
import numpy as np

def skeleton_graph(skel):
    """Convertit le squelette en graphe NetworkX pour calculer la longueur"""
    coords = np.argwhere(skel)
    idx_map = { (int(y),int(x)): i for i,(y,x) in enumerate(coords) }
    G = nx.Graph()
    for (y,x), i in idx_map.items():
        G.add_node(i, pos=(y,x))
        for dy in (-1,0,1):
            for dx in (-1,0,1):
                if dy==0 and dx==0: continue
                ny, nx_ = y+dy, x+dx
                if (ny,nx_) in idx_map:
                    j = idx_map[(ny,nx_)]
                    dist = math.hypot(dy, dx)
                    G.add_edge(i,j,weight=dist)
    return G, coords

def find_endpoints(G):
    """Trouve les points terminaux (degré = 1) du graphe"""
    return [n for n,d in G.degree() if d==1]

def longest_path_length(G):
    """Calcule la longueur du chemin le plus long dans le graphe"""
    endpoints = find_endpoints(G)
    if not endpoints:
        return sum(d['weight'] for u,v,d in G.edges(data=True))/2
    maxlen = 0.0
    for i in range(len(endpoints)):
        lengths = nx.single_source_dijkstra_path_length(G, endpoints[i], weight='weight')
        for j in endpoints[i+1:]:
            if j in lengths and lengths[j] > maxlen:
                maxlen = lengths[j]
    return maxlen

## 7. Conversion de la Longueur en Centimètres

In [19]:
def pixels_to_cm(longueur_pixels, longueur_objet_reel_cm, longueur_objet_pixels):
    """Convertit des pixels en cm en utilisant un objet de référence"""
    pixel_size_cm = longueur_objet_reel_cm / longueur_objet_pixels
    return longueur_pixels * pixel_size_cm

## 8. Pipeline Complet : Traitement des Masques et Calcul des Longueurs

In [20]:
# --- Pipeline complet ---
train_masks_folder = "/content/dataset_masks/train"

# Paramètres de calibration (à ajuster selon vos images)
# Exemple : un câble de référence mesure 10 cm et fait 200 pixels sur l'image
LONGUEUR_OBJET_REEL_CM = 10
LONGUEUR_OBJET_PIXELS = 200

for mask_file in os.listdir(train_masks_folder):
    mask_path = os.path.join(train_masks_folder, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is None:
        print(f"Fichier introuvable : {mask_path}")
        continue

    # Appliquer le pipeline
    mask_clean = preprocess_mask(mask)
    skel = skeletonize_mask(mask_clean)
    G, coords = skeleton_graph(skel)
    longueur_pixels = longest_path_length(G)

    # Conversion en cm
    longueur_cm = pixels_to_cm(longueur_pixels, LONGUEUR_OBJET_REEL_CM, LONGUEUR_OBJET_PIXELS)

    print(f"{mask_file} : longueur = {longueur_pixels:.2f} pixels ≈ {longueur_cm:.2f} cm")

photo-1761661842583_jpg.rf.4196021dd4f0a2bcf48a2cd1a1057894_mask.png : longueur = 409.12 pixels ≈ 20.46 cm
photo-1761661127308_jpg.rf.984bde6a4b8f7722e1aa9b169b2458e9_mask.png : longueur = 937.08 pixels ≈ 46.85 cm
photo-1761661263657_jpg.rf.e2e9c5b94a13646dccf938f3adfac8f0_mask.png : longueur = 1603.57 pixels ≈ 80.18 cm
photo-1761661592850_jpg.rf.2275af26b8f9458c212df0f8f22036ef_mask.png : longueur = 693.04 pixels ≈ 34.65 cm
photo-1761661736171_jpg.rf.0f602cf08ac62cc9584612bf542be3d7_mask.png : longueur = 1437.96 pixels ≈ 71.90 cm
photo-1761661326741_jpg.rf.6157e39646375ba6620a713d10f66270_mask.png : longueur = 864.78 pixels ≈ 43.24 cm
IMG_0878_JPG.rf.9f9dda1bd7f148657739f570bba6551a_mask.png : longueur = 783.18 pixels ≈ 39.16 cm
photo-1761661699561_jpg.rf.74f4276e01a2ccf7abfe3d23c27b736c_mask.png : longueur = 1400.70 pixels ≈ 70.03 cm
IMG_0881_JPG.rf.4ffbf7ca2179ab3f0390119a5361f280_mask.png : longueur = 1228.80 pixels ≈ 61.44 cm
IMG_0879_JPG.rf.9c8d15760f01f367842f9203c85d4f12_mask.p

In [21]:
import matplotlib.pyplot as plt

# Critères de qualité
LONGUEUR_MIN_OK = 22  # cm
LONGUEUR_MAX_OK = 60  # cm

# --- Pipeline avec affichage ---
for mask_file in os.listdir(train_masks_folder):
    mask_path = os.path.join(train_masks_folder, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is None:
        print(f"Fichier introuvable : {mask_path}")
        continue

    # Appliquer le pipeline
    mask_clean = preprocess_mask(mask)
    skel = skeletonize_mask(mask_clean)
    G, coords = skeleton_graph(skel)
    longueur_pixels = longest_path_length(G)
    longueur_cm = pixels_to_cm(longueur_pixels, LONGUEUR_OBJET_REEL_CM, LONGUEUR_OBJET_PIXELS)

    # Déterminer si le câble est OK ou DÉFECTUEUX
    if LONGUEUR_MIN_OK <= longueur_cm <= LONGUEUR_MAX_OK:
        statut = "OK"
        couleur_titre = "green"
    else:
        statut = "DÉFECTUEUX"
        couleur_titre = "red"

    # --- Affichage ---
    plt.figure(figsize=(8,8))
    plt.imshow(mask, cmap='gray')
    plt.imshow(skel, cmap='hot', alpha=0.7)  # superpose le squelette en rouge
    plt.title(f"{mask_file}\nLongueur = {longueur_pixels:.2f} pixels ≈ {longueur_cm:.2f} cm\nStatut: {statut}",
              color=couleur_titre, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()

    print(f"{mask_file} : {longueur_cm:.2f} cm - {statut}")

Output hidden; open in https://colab.research.google.com to view.